### Dataset_Preparation

In [1]:
import os
import random
import shutil

splitsize = 0.85
categories = []

source_folder = '/Users/arahjou/Downloads/U_NET_GI_Segmentation/data'
folders = os.listdir(source_folder)
print(folders)

for subfolder in folders:
    if os.path.isdir(os.path.join(source_folder, subfolder)):
        categories.append(subfolder)

categories.sort()
print(categories)

target_folder = "/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model"
if not os.path.exists(target_folder):
    os.mkdir(target_folder)

trainPath = os.path.join(target_folder, "train")
validatePath = os.path.join(target_folder, "validation")

if not os.path.exists(trainPath):
    os.mkdir(trainPath)
if not os.path.exists(validatePath):
    os.mkdir(validatePath)

def split_data(SOURCE, TRAINING, VALIDATION, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = os.path.join(SOURCE, filename)
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(f"{filename} is 0 length, ignoring it ...")
    print(len(files))

    trainingLength = int(len(files) * SPLIT_SIZE)
    shuffleSet = random.sample(files, len(files))
    trainingSet = shuffleSet[:trainingLength]
    validSet = shuffleSet[trainingLength:]

    for filename in trainingSet:
        thisFile = os.path.join(SOURCE, filename)
        destination = os.path.join(TRAINING, filename)
        shutil.copyfile(thisFile, destination)

    for filename in validSet:
        thisFile = os.path.join(SOURCE, filename)
        destination = os.path.join(VALIDATION, filename)
        shutil.copyfile(thisFile, destination)

for category in categories:
    trainDestPath = os.path.join(trainPath, category)
    validateDestPath = os.path.join(validatePath, category)

    if not os.path.exists(trainDestPath):
        os.mkdir(trainDestPath)
    if not os.path.exists(validateDestPath):
        os.mkdir(validateDestPath)

    sourcePath = os.path.join(source_folder, category)
    print(f"Copying from: {sourcePath} to: {trainDestPath} and {validateDestPath}")
    split_data(sourcePath, trainDestPath, validateDestPath, splitsize)


['.DS_Store', 'class_II', 'class_III', 'class_I']
['class_I', 'class_II', 'class_III']
Copying from: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/class_I to: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/train/class_I and /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation/class_I
14286
Copying from: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/class_II to: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/train/class_II and /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation/class_II
16545
Copying from: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/class_III to: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/train/class_III and /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation/class_III
7663


## Preparing_Model

In [2]:
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

trainPath = "/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/train"
validPath = "/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation"

trainGenerator = ImageDataGenerator(
    rotation_range=15, width_shift_range=0.1,
    height_shift_range=0.1, brightness_range=(0.8, 1.2)
).flow_from_directory(trainPath, target_size=(320, 320), batch_size=32)

validGenerator = ImageDataGenerator(
    rotation_range=15, width_shift_range=0.1,
    height_shift_range=0.1, brightness_range=(0.8, 1.2)
).flow_from_directory(validPath, target_size=(320, 320), batch_size=32)

# Build the model
baseModel = MobileNetV3Large(weights="imagenet", include_top=False)

x = baseModel.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)

predictionLayer = Dense(3, activation='softmax')(x)

model = Model(inputs=baseModel.input, outputs=predictionLayer)

# freezing the layers of the MobileNetV3 (already trained)

for layer in model.layers[:-5]:
    layer.trainable = False

# Compile
optimizer = Adam(learning_rate=0.0001)  # Correct argument name is `learning_rate`
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

# Train
model.fit(trainGenerator, validation_data=validGenerator, epochs=5)

# Save the model
modelSavedPath = "/Users/arahjou/Downloads/U_NET_GI_Segmentation/model/MobileNetV3.keras"
model.save(modelSavedPath)

Found 32719 images belonging to 3 classes.
Found 5775 images belonging to 3 classes.


/Users/arahjou/Documents/Com_Conda/.conda/lib/python3.11/site-packages/keras/src/applications/mobilenet_v3.py:512: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(
2024-05-08 20:38:05.411309: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-05-08 20:38:05.411329: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-05-08 20:38:05.411335: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-05-08 20:38:05.411365: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-08 20:38:05.411375: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:

Epoch 1/5


2024-05-08 20:38:07.703426: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Users/arahjou/Documents/Com_Conda/.conda/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1023/1023 ━━━━━━━━━━━━━━━━━━━━ 634s 615ms/step - accuracy: 0.4138 - loss: 1.0712 - val_accuracy: 0.4329 - val_loss: 1.0576
Epoch 2/5
1023/1023 ━━━━━━━━━━━━━━━━━━━━ 348s 338ms/step - accuracy: 0.4313 - loss: 1.0549 - val_accuracy: 0.4289 - val_loss: 1.0535
Epoch 3/5
1023/1023 ━━━━━━━━━━━━━━━━━━━━ 348s 339ms/step - accuracy: 0.4366 - loss: 1.0508 - val_accuracy: 0.4376 - val_loss: 1.0524
Epoch 4/5
1023/1023 ━━━━━━━━━━━━━━━━━━━━ 350s 340ms/step - accuracy: 0.4375 - loss: 1.0482 - val_accuracy: 0.4395 - val_loss: 1.0536
Epoch 5/5
1023/1023 ━━━━━━━━━━━━━━━━━━━━ 354s 344ms/step - accuracy: 0.4413 - loss: 1.0457 - val_accuracy: 0.4324 - val_loss: 1.0517


## Test the model

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from PIL import Image
import cv2

# get the list of categories
categories = os.listdir('/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/train')
categories.sort()
print(categories)

# load the saved model
modelSavedPath = "/Users/arahjou/Downloads/U_NET_GI_Segmentation/model/MobileNetV3.keras"
model = tf.keras.models.load_model(modelSavedPath)

# predict the image
def classify_image(imageFile):
    img = load_img(imageFile)  # Use load_img for consistency with Keras
    img = img.resize((320, 320), Image.Resampling.LANCZOS)  # Use Resampling for resizing

    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)

    print(x.shape)
    pred = model.predict(x)
    print(pred)

    # Get the highest prediction value
    categoryValue = np.argmax(pred, axis=1)
    categoryValue = categoryValue[0]

    print(pred)

    result = categories[categoryValue]
    return result

image_path = "/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation/class_III/slice_0015_360_310_1.50_1.50.png"
resultText = classify_image(image_path)
print(resultText)

image = cv2.imread(image_path)
image = cv2.putText(image, resultText, (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)  # Use white color for text for visibility
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


['class_I', 'class_II', 'class_III']


FileNotFoundError: [Errno 2] No such file or directory: '/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation/class_III/slice_0015_360_310_1.50_1.50.png'